In [1]:
!pip install bsedata

In [2]:
from bsedata import bse
import pandas as pd
import numpy as np

# Object which implements functionality of BSE Package
b = bse.BSE()
# Sorted List Codes of Top Gainers
b.topGainers()

In [3]:
# Details of Top Gainers
top_gainers = b.topGainers()
res = []
for i,gainer in enumerate(top_gainers):
  try:
    res.append( b.getQuote(gainer['scripCode']) )
  except:
    pass

In [4]:
# Details of Top Gainers
top_gainers = b.topGainers()
top_gainer_names = []
for gainer in top_gainers:
  try:
    top_gainer_names.append( b.getQuote(gainer['scripCode'])['companyName'] )
  except:
    pass


In [12]:
# Getting the scripCodes and corresponding company names
b.updateScripCodes()
comps = b.getScripCodes()
comps

In [6]:
d = {
    'scripCode': [],
    'companyName': [],
}
for code,name in comps.items():
  d['scripCode'].append(code)
  d['companyName'].append(name)
companies_df = pd.DataFrame(d)
companies_df.to_csv('companies.csv', index=None)

In [7]:
# Creating Custom Thread to download Stock Data

from threading import Thread
from time import time
import math

class GetQuoteThread(Thread):

  def __init__(self, i, comps):
    # print(f'Init: {i}')
    Thread.__init__(self)
    self.data = dict()
    self.comps = comps
    self.i = i
    self.success = 0
    self.fail = 0

  def run(self):
    for code in self.comps:

      try:
        self.data[code] = b.getQuote(code)
        self.success += 1
      except Exception:
        self.fail += 1


In [8]:
# Takes around 3 mins
# To reduce this, decrease stocks per_thread -> Increases no: of Threads

threads = []
codes = list( comps.keys() )
per_thread = 125

for i in range(per_thread, per_thread*math.ceil(len(codes)//per_thread)+1, per_thread):
  threads.append( GetQuoteThread( i, codes[i-per_thread:i] ) )

for thread in threads:
  thread.start()

comp_quotes = dict()
s,f = 0,0
for thread in threads:
  thread.join()
  comp_quotes.update(thread.data)
  s += thread.success
  f += thread.fail
  # print(thread.t2-thread.t1)
  # print(thread.data)

In [15]:
# Just some debugging -> Works alright!
# Some Threads have failed. Maybe some outdated stocks
len(comp_quotes), s, f, s+f

In [16]:
# Picking out the Top 'n' companies in terms of their MarketCap
n = 1000
codes = list( comp_quotes.keys() )
codes.sort(key = lambda x: comp_quotes[x]['marketCapFull'])
codes = codes[:n]


In [17]:
# Getting The Price data for the past 12 months
prices = {}
for code in codes:
  ret = b.getPeriodTrend( code, '12M' )
  # print(code, ret)
  prices[code] = [ day['value'] for day in ret ]
  # print(len(data[comp]))be


In [18]:
# Pick only the stocks where data isn't missing
working_days = max(len(prices[code]) for code in codes)
price_data = dict()
final_codes = []

for code in codes:
  if len(prices[code]) == working_days:
    price_data[comps[code]] = prices[code]
    final_codes.append(code)

prices_df = pd.DataFrame(price_data)
prices_df.to_csv('price_data.csv', index=None)

In [ ]:
prices_df = pd.read_csv('price_data.csv')

In [ ]:
# Price Data of Prev 12 Months
prices_df

In [ ]:
# Daily Returns for past 12 Months
ret_df = prices_df.pct_change().drop(0)
ret_df

In [ ]:
# Covariance Matrix of Returns Matrix
cov = np.round( ret_df.cov(), 6 )
cov